[View in Colaboratory](https://colab.research.google.com/github/kamalkarki/mnist/blob/master/Tf_exercise.ipynb)

In [0]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [3]:
def get_data():
  mnist_download = fetch_mldata('MNIST original')
  mnist_data = {} # dictionay for dataset
  X_train, X_test, y_train, y_test = train_test_split( mnist_download.data, mnist_download.target, test_size=0.20, random_state=42)
  print('Data for the mnist data has been downloaded and split into Train and Test data with size:')
  print('X_train shape' + str(X_train.shape))
  print('y_train shape' + str(y_train.shape))
  print('y_test shape' + str(y_test.shape))
  print('X_test shape' + str(X_test.shape))
  mnist_data['X_train'] = X_train
  mnist_data['X_test'] = X_test
  mnist_data['y_train'] = y_train
  mnist_data['y_test'] = y_test
  return mnist_data
  
mnist_data = get_data()  

Data for the mnist data has been downloaded and split into Train and Test data with size:
X_train shape(56000, 784)
y_train shape(56000,)
y_test shape(14000,)
X_test shape(14000, 784)


In [0]:
def reshaping(data):
  # reshaping X
  #X_train = np.reshape(X_train,(X_train.shape[0],28,28))
  #X_test = np.reshape(X_test,(X_test.shape[0],28 ,28))
  # reshaping the y
  data['y_train'] = np.reshape(data['y_train'],(data['y_train'].shape[0],-1))
  data['y_test'] = np.reshape(data['y_test'],(data['y_test'].shape[0],-1))
  return data
mnist_data = reshaping(mnist_data)  

In [0]:
def hot_encoding(data):
  # 1. INSTANTIATE
  enc = OneHotEncoder()
  # 2. FIT
  enc.fit(data)
  # 3. Transform
  onehotlabels = enc.transform(data).toarray()
  #onehotlabels.shape
  return onehotlabels

mnist_data['y_train'] = hot_encoding(mnist_data['y_train'])
mnist_data['y_test'] = hot_encoding(mnist_data['y_test'])

In [0]:
# making the graph
height = 28
width = 28
channels = 1
n_inputs = height * width

tf.reset_default_graph()

with tf.name_scope("inputs"):
  X = tf.placeholder(tf.float32, shape=[None, height * width], name="X")
  X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
  y = tf.placeholder(tf.int32, shape=[None,10], name="y")

#layer 1  
conv1 = tf.layers.conv2d(X_reshaped, filters= 12, kernel_size= 3,
                         strides=1, padding="SAME",
                         activation=tf.nn.relu, name="conv1")


  
#layer 2  
conv2 = tf.layers.conv2d(conv1, filters = 12, kernel_size = 3,
                         strides= 1, padding = "SAME",
                         activation=tf.nn.relu, name="conv2")

with tf.name_scope("pool1"):
  #pooling
  pool1 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
  


with tf.name_scope("fc1"):
  #flat
  flat = tf.contrib.layers.flatten(pool1)
  #fully connected
  fc = tf.contrib.layers.fully_connected(flat, 64, activation_fn=None)
  fc1_drop = tf.layers.dropout(fc,0.5)
  
with tf.name_scope("fc2"):
  #fully connected 2
  fc = tf.contrib.layers.fully_connected(flat, 10, activation_fn=None)  

# prediction and loss calculation in the same scope
with tf.name_scope("training"):
  xentropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=fc, labels= y)
  loss = tf.reduce_mean(xentropy)
  optimizer = tf.train.AdamOptimizer()
  training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
  #correct = tf.nn.in_top_k(logits, y, 1)
  #accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
  # Calculate the correct predictions
  predict_op = tf.argmax(fc, 1)
  correct_prediction = tf.equal(predict_op, tf.argmax(y, 1))

  # Calculate accuracy on the test set
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
  
def fetch_batch(epoch, batch_index, batch_size,data):
    np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size)  
    X_train = data['X_train']
    y_train = data['y_train']
    X_batch = X_train[indices] 
    y_batch = y_train[indices] 
    return X_batch, y_batch

In [11]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
#session = tf.Session(config=config, ...)
!mkdir('/model')
n_epochs = 50
batch_size = 1000
m = mnist_data['X_train'].shape[0]
n_batches = int(np.ceil(m / batch_size))

init = tf.global_variables_initializer()
save = tf.train.Saver()

with tf.Session(config=config) as sess:
  init.run()
  for epoch in range(0,n_epochs):
    for batch_index in range(0,n_batches):
      X_train,y_train = fetch_batch(epoch, batch_index, batch_size,mnist_data)
      sess.run(training_op, feed_dict={X: X_train, y: y_train})
    if epoch % 5 == 0:  
      acc_train = accuracy.eval(feed_dict={X: X_train, y: y_train})
      acc_test = accuracy.eval(feed_dict={X: mnist_data['X_test'], y: mnist_data['y_test']})
      print("Epoch number:",epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
      
  save.save(sess,'model/cnn_1.ckpt')    

/bin/bash: -c: line 0: syntax error near unexpected token `'/model''
/bin/bash: -c: line 0: `mkdir('/model')'
Epoch number: 0 Train accuracy: 0.909 Test accuracy: 0.8974286
Epoch number: 5 Train accuracy: 0.977 Test accuracy: 0.963
Epoch number: 10 Train accuracy: 0.989 Test accuracy: 0.97314286
Epoch number: 15 Train accuracy: 0.995 Test accuracy: 0.97585714
Epoch number: 20 Train accuracy: 0.998 Test accuracy: 0.97814286
Epoch number: 25 Train accuracy: 0.999 Test accuracy: 0.9767143
Epoch number: 30 Train accuracy: 1.0 Test accuracy: 0.97742856
Epoch number: 35 Train accuracy: 1.0 Test accuracy: 0.9779286
Epoch number: 40 Train accuracy: 1.0 Test accuracy: 0.9785714
Epoch number: 45 Train accuracy: 1.0 Test accuracy: 0.97814286


## Task 2

In [0]:
!mkdir data